In [1]:

import pandas as pd
from scipy import sparse 
from typing import Optional
import numpy as np
import random as rnd
from sklearn.preprocessing import MinMaxScaler
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# week 1: member 2

data = pd.read_excel(r"C:\Users\kidig\OneDrive\Desktop\dsa2040group\sales_and_customer_insights_dirty.xlsx")
print(data.head())



# View basic info
print("\n Dataset Info:")
print(data.info())

print("\n First 5 Rows:")
print(data.head())

print("\n Missing Values Count:")
print(data.isnull().sum())

print("\n Duplicate Rows:", data.duplicated().sum())

print("\n Summary Statistics:")
print(data.describe(include='all'))




# --- Step 3: Check for negative or zero quantities and prices ---
negative_qty = data[data['Quantity'] <= 0]
negative_price = data[data['UnitPrice'] <= 0]

print(f"\n Rows with non-positive quantities: {len(negative_qty)}")
print(f" Rows with non-positive unit prices: {len(negative_price)}")

# --- Step 4: Check country name inconsistencies ---
print("\n Unique countries:\n", data['Country'].unique())

# --- Step 5: Quick check for outliers using describe() ---
print("\n Summary statistics for numeric columns:\n", data.describe())


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER       6.0   
1    536365     71053                  WHITE METAL LANTERN       6.0   
2       NaN    84406B       CREAM CUPID HEARTS COAT HANGER       8.0   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE       6.0   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.       6.0   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0             NaN  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  

 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547328 entries, 0 to 547327
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------  